In [1]:
%load_ext autoreload
%autoreload 2
import keras

Using TensorFlow backend.


In [2]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, InputLayer
from keras.layers.convolutional import Conv2D, MaxPooling2D
import sys
import os
pwd = os.getcwd()
sys.path.append("/root/Gan/jidian/MLexperiments")
sys.path.append("/root/Gan/jidian")
sys.path.append(pwd)
sys.path.append(os.path.dirname(os.getcwd()))
import MLexperiments.classes
from MLexperiments.classes import ReadAutoLabeledData
import tensorflow as tf
import MLexperiments.config.parameters
from keras.layers import LSTM, Reshape
import matplotlib.pyplot as plt
import utils
from keras.callbacks import ModelCheckpoint
import os
from keras.callbacks import TensorBoard
from time import time
from keras.models import Model
from keras.layers import Input, Dense
import datetime
import Models
import os, errno

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3,4"
#+12h = chinese time zone


#
tf.flags.DEFINE_integer("BATCH_SIZE",128, "size")
tf.flags.DEFINE_integer("MODEL_TYPE",0 , "0: CNN, 1:RNN")
tf.flags.DEFINE_string("CHECKPOINTPATH",'logs/{}'.format(datetime.datetime.fromtimestamp(time()).strftime('%Y-%m-%d %H:%M:%S'))\
                       , "Path to save the best models")
tf.flags.DEFINE_string("MODE",  1\
                       , " mode is data source specification;   mode: 1 => 3 human + rain: 1 means high only, 2 high + mid, 3 high+mid+low ,4=>6 man + rain + wind:      4 means high only, 5 high + mid, 6 high+mid+low , 7 => qinshang + factory for testing")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()

['-f', '/run/user/0/jupyter/kernel-7d2a8c9f-016a-4964-a153-035c3fe620bc.json']

In [4]:
#data reading and initialization

try:
    os.makedirs(FLAGS.CHECKPOINTPATH)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise


In [5]:
myDataSet = utils.read_data_sets(one_hot=True, test_size = 0.1, validation_size = 0.1, fake_data = False\
                                #todo[][]   , mode = FLAGS.MODE
                                )

(X_train, y_train), (X_test, y_test) , (X_Ftest, y_Ftest)= (myDataSet.train.images, myDataSet.train.labels), (myDataSet.validation.images, myDataSet.validation.labels), (myDataSet.test.images, myDataSet.test.labels)

print(X_train[0].shape)
print(y_train[0])

X_train = X_train.reshape(X_train.shape[0], MLexperiments.config.parameters.SAMPLE_LEN, MLexperiments.config.parameters.SAMPLE_HEIGHT, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], MLexperiments.config.parameters.SAMPLE_LEN, MLexperiments.config.parameters.SAMPLE_HEIGHT, 1).astype('float32')
X_Ftest = X_Ftest.reshape(X_Ftest.shape[0], MLexperiments.config.parameters.SAMPLE_LEN, MLexperiments.config.parameters.SAMPLE_HEIGHT, 1).astype('float32')

#X_train /= 255
#X_test /= 255


def tran_y(y):
    y_ohe = np.zeros(MLexperiments.config.parameters.OUTPUTNUM)
    
    y_ohe[int(y)] = 1 
    return y_ohe

# def tran_y(y):
#     y_ohe = np.zeros(MLexperiments.config.parameters.OUTPUTNUM)
#     if y:
#         y_ohe[1] = 1
#     else:
#         y_ohe[0] = 1
#     return y_ohe
# def tran_y(y):
#     return y
#import pdb;pdb.set_trace()
y_train_ohe = np.array([tran_y(y_train[i]) for i in range(len(y_train))])
y_test_ohe = np.array([tran_y(y_test[i]) for i in range(len(y_test))])

> /root/Gan/jidian/MLexperiments/scripts/utils.py(129)read_data_sets()
-> np_input, labelData = np.concatenate((man_input, rain_input)), np.concatenate((man_label, rain_label))
(Pdb) c
number of each type (balanced):
15304
total (balanced):
30608
 np_input shape:  (30608, 100, 10) labelData shape:  (30608,)
read succesful
(1000,)
0.0
--Return--
> <ipython-input-5-eca442341593>(32)<module>()->None
-> import pdb;pdb.set_trace()
(Pdb) c


In [ ]:
model = Models.getMODEL(FLAGS.MODEL_TYPE)
model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
lossHistory = LossHistory()
checkpointer = ModelCheckpoint(filepath = os.path.join(FLAGS.CHECKPOINTPATH, 'best_weights.hdf5'), verbose=1, save_best_only=True)
tensorboard = TensorBoard(log_dir=FLAGS.CHECKPOINTPATH,histogram_freq=0, batch_size=FLAGS.BATCH_SIZE, write_graph=True, write_grads=False, \
                          write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

model.summary()
#



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 8, 64)         640       
_________________________________________________________________
dropout_1 (Dropout)          (None, 98, 8, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 6, 128)        73856     
_________________________________________________________________
dropout_2 (Dropout)          (None, 96, 6, 128)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 96, 6, 32)         4128      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 94, 4, 32)         9248      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 92, 2, 16)         4624      
__________

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "8,7"

history = model.fit(X_train, y_train_ohe, validation_data=(X_test, y_test_ohe), epochs=200, batch_size=FLAGS.BATCH_SIZE, \
                callbacks=[\
                           lossHistory,\
                           checkpointer,\
                           tensorboard\
                          ] )

Train on 24793 samples, validate on 2754 samples
Epoch 1/200
24793/24793 [==============================] - 102s - loss: 0.6235 - acc: 0.6452 - val_loss: 0.2522 - val_acc: 0.8914
Epoch 2/200
24793/24793 [==============================] - 100s - loss: 0.2392 - acc: 0.8972 - val_loss: 0.1549 - val_acc: 0.9397
Epoch 3/200
24793/24793 [==============================] - 100s - loss: 0.1670 - acc: 0.9340 - val_loss: 0.0977 - val_acc: 0.9666
Epoch 4/200
24793/24793 [==============================] - 100s - loss: 0.1118 - acc: 0.9620 - val_loss: 0.0793 - val_acc: 0.9735
Epoch 5/200
24793/24793 [==============================] - 101s - loss: 0.0815 - acc: 0.9741 - val_loss: 0.0695 - val_acc: 0.9779
Epoch 6/200
24793/24793 [==============================] - 100s - loss: 0.0659 - acc: 0.9793 - val_loss: 0.0507 - val_acc: 0.9847
Epoch 7/200
24793/24793 [==============================] - 101s - loss: 0.0562 - acc: 0.9824 - val_loss: 0.0419 - val_acc: 0.9869
Epoch 8/200
24793/24793 [================

24793/24793 [==============================] - 99s - loss: 0.0090 - acc: 0.9974 - val_loss: 0.0098 - val_acc: 0.9960
Epoch 29/200
24793/24793 [==============================] - 99s - loss: 0.0075 - acc: 0.9977 - val_loss: 0.0066 - val_acc: 0.9967
Epoch 30/200
24793/24793 [==============================] - 101s - loss: 0.0064 - acc: 0.9981 - val_loss: 0.0083 - val_acc: 0.9975
Epoch 31/200
24793/24793 [==============================] - 100s - loss: 0.0078 - acc: 0.9972 - val_loss: 0.0480 - val_acc: 0.9851
Epoch 32/200
24793/24793 [==============================] - 99s - loss: 0.0071 - acc: 0.9977 - val_loss: 0.0095 - val_acc: 0.9967
Epoch 33/200
24793/24793 [==============================] - 100s - loss: 0.0064 - acc: 0.9979 - val_loss: 0.0034 - val_acc: 0.9996
Epoch 34/200
24793/24793 [==============================] - 99s - loss: 0.0155 - acc: 0.9948 - val_loss: 0.0054 - val_acc: 0.9982
Epoch 35/200
24793/24793 [==============================] - 100s - loss: 0.0050 - acc: 0.9984 - val_

24793/24793 [==============================] - 56s - loss: 0.0022 - acc: 0.9993 - val_loss: 0.0061 - val_acc: 0.9989
Epoch 61/200
24793/24793 [==============================] - 56s - loss: 0.0017 - acc: 0.9995 - val_loss: 0.0048 - val_acc: 0.9993
Epoch 62/200
24793/24793 [==============================] - 56s - loss: 0.0017 - acc: 0.9995 - val_loss: 0.0042 - val_acc: 0.9996
Epoch 63/200
24793/24793 [==============================] - 56s - loss: 0.0025 - acc: 0.9990 - val_loss: 0.0038 - val_acc: 0.9993
Epoch 64/200
24793/24793 [==============================] - 56s - loss: 0.0017 - acc: 0.9995 - val_loss: 0.0031 - val_acc: 0.9989
Epoch 65/200
24793/24793 [==============================] - 56s - loss: 0.0014 - acc: 0.9996 - val_loss: 0.0029 - val_acc: 0.9993
Epoch 66/200
24793/24793 [==============================] - 56s - loss: 0.0018 - acc: 0.9995 - val_loss: 0.0133 - val_acc: 0.9964
Epoch 67/200
24793/24793 [==============================] - 56s - loss: 0.0013 - acc: 0.9996 - val_loss

In [ ]:

# summarize history for accuracy
plt.plot(lossHistory.acc)
plt.plot(lossHistory.val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('batch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(lossHistory.losses)
plt.plot(lossHistory.val_losses)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('batch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

scores = model.evaluate(X_test, y_test_ohe, verbose=0)

In [ ]:
scores

In [ ]:

for index in range(10):
    print("expected: "+str( y_Ftest[index]))
    prediction = model.predict(np.expand_dims(X_Ftest[index], axis=0))[0]
    print("prediction:" + str(np.argmax(prediction,axis = 0)))
    time_series = X_Ftest.reshape(X_Ftest.shape[0], MLexperiments.config.parameters.SAMPLE_LEN, MLexperiments.config.parameters.SAMPLE_HEIGHT) \
        .astype('float32')[index]
    # plt.imshow( X_train.reshape(X_train.shape[0], MLexperiments.config.parameters.SAMPLE_LEN, MLexperiments.config.parameters.SAMPLE_HEIGHT).astype('float32')[0]);
    # plt.colorbar()
    # plt.show()
    plt.plot(time_series.T[0])
    plt.plot(time_series.T[1])
    plt.plot(time_series.T[2])
    plt.plot(time_series.T[3])
    plt.plot(time_series.T[4])
    plt.plot(time_series.T[5])
    plt.plot(time_series.T[6])
    plt.plot(time_series.T[7])
    plt.plot(time_series.T[8])
    plt.plot(time_series.T[9])
    plt.show()
   

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# datagen = ImageDataGenerator(
#        vertical_flip=False,
#         horizontal_flip=True)

# index = 0
# time_series = X_test.reshape(X_test.shape[0], MLexperiments.config.parameters.SAMPLE_LEN, MLexperiments.config.parameters.SAMPLE_HEIGHT) \
#         .astype('float32')[index]

# # plt.plot(time_series.T[0])
# # plt.show()    
# # plt.plot(time_series.T[1])
# # plt.show()    
# # plt.plot(time_series.T[2])
# # plt.show()    
# # plt.plot(time_series.T[3])
# # plt.show()    
# # plt.plot(time_series.T[4])
# # plt.show()    
# # plt.plot(time_series.T[5])
# # plt.show()    
# # print("+++++++++++++++++++++++++++++++++++++++++++++")    
    
    
# # img = load_img('data/train/cats/cat.0.jpg')  # this is a PIL image
# # x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
# # x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
# x = time_series.reshape((1,)+time_series.shape + (1,))
# print("x.shape  ")
# print(x.shape)
# # the .flow() command below generates batches of randomly transformed images
# # and saves the results to the `preview/` directory
# i = 0
# for batch in datagen.flow(x, batch_size=1):
#     i += 1
#     time_series = batch[0]
#     time_series = time_series.reshape( MLexperiments.config.parameters.SAMPLE_LEN, MLexperiments.config.parameters.SAMPLE_HEIGHT) \
#         .astype('float32')
#     plt.plot(time_series.T[0])
#     plt.plot(time_series.T[1])
#     plt.plot(time_series.T[2])
#     plt.plot(time_series.T[3])
#     plt.plot(time_series.T[4])
#     plt.plot(time_series.T[5])
#     plt.show()
# #     plt.plot(time_series.T[0])
# #     plt.show()    
# #     plt.plot(time_series.T[1])
# #     plt.show()    
# #     plt.plot(time_series.T[2])
# #     plt.show()    
# #     plt.plot(time_series.T[3])
# #     plt.show()    
# #     plt.plot(time_series.T[4])
# #     plt.show()    
# #     plt.plot(time_series.T[5])
# #     plt.show()    
# #     print("+++++++++++++++++++++++++++++++++++++++++++++")    
    
#     if i > 3:
#         break  # otherwise the generator would loop indefinitely